In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos['denominacion_empresa']=avisos['denominacion_empresa'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16501974


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
8280428,1112242436,MVOzmw3,1,FEM,35.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,True,False,True,NaN
4800684,1111661549,ZZzwb1,0,MASC,34.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
12008504,1112348604,X95pl48,1,NO_DECLARA,200.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,True,NaN
15567414,1112426916,lD2pAX3,0,FEM,32.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
4935565,1112289833,vVjARJP,0,FEM,18.0,-,-,Graduado,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
3525382,1112317212,GNZRd2p,1,FEM,20.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
8947517,1112264475,8MawAjD,0,FEM,26.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,True,False,True,NaN
9342555,1112334437,86Bqrx,0,FEM,33.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
15527983,1111397360,2zQd2v1,0,FEM,25.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,NaN
313846,1112336456,GNZ6EAp,0,NO_DECLARA,200.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
11655176,1112512069,8bPYal,0,FEM,37.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,True,False,True,0.000000
1326134,1112281808,pzMzDvM,0,FEM,28.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.000000
3872250,1112362015,YjVa9AD,0,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
7428572,1112256011,mzGkkKd,1,FEM,28.0,-,-,Graduado,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.000000
6964229,1112207868,YR6rbP,0,MASC,34.0,-,-,-,-,-,...,True,False,False,False,False,False,False,False,False,0.000000
3380190,1112400775,zvaeDm5,1,FEM,21.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.108108
7058013,1112384651,96MGQYa,1,MASC,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
12059092,1112463713,JBxvRA3,1,MASC,29.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.011236
4307667,1112392539,qewv2Bw,1,MASC,25.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
11401111,1111982422,PmJvowZ,0,FEM,43.0,-,-,-,-,-,...,False,False,False,True,False,False,False,False,False,0.000000


In [15]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Administración,2
1,0z5Dmrd,Almacén / Depósito / Expedición,1
2,0z5Dmrd,Atención al Cliente,1
3,0z5Dmrd,Auditoría,1
4,0z5Dmrd,Comercial,1


In [16]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,-,1
1,0z5Dmrd,Gerencia / Alta Gerencia / Dirección,2
2,0z5Dmrd,Junior,4
3,0z5Dmrd,Otro,1
4,0z5Dmrd,Senior / Semi-Senior,17


In [17]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,23
1,0z5Dmrd,Part-time,1
2,0z5Dmrd,Teletrabajo,1
3,0z5JW1r,Full-time,28
4,0z5JW1r,Part-time,1


In [18]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
4173965,1112349890,GNJBWxp,1,MASC,43.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.000000,3,31,15
9472399,1112447961,A3XpJ3j,0,MASC,34.0,-,-,-,-,-,...,False,True,True,False,False,False,0.000000,0,25,6
6688721,1112185932,bOVj699,1,MASC,29.0,-,-,Graduado,-,Graduado,...,False,True,True,False,False,False,0.000000,0,78,16
2962499,1111903673,KBaQ150,1,FEM,20.0,-,-,-,-,-,...,False,False,False,False,True,False,0.109890,2,38,6
7599395,1112320618,6rQEmp3,1,FEM,26.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.000000,0,31,3
10110561,1112204743,ak5l83W,1,MASC,21.0,-,-,-,-,Graduado,...,False,False,False,True,False,True,0.000000,15,34,30
10424452,1112444569,QNMPxE5,1,FEM,25.0,-,-,-,-,-,...,False,False,False,False,False,False,0.001464,57,585,440
11859090,1112391090,8KlKKO,1,MASC,37.0,-,-,En Curso,Graduado,Graduado,...,False,False,False,False,True,False,0.000000,16,63,10
9234854,1112366118,8mEoYj,0,FEM,27.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.000000,1,28,6
10642598,1112129870,LNwrJBr,1,FEM,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.000000,56,376,29


In [19]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0

Train:  15281843 Test:  804308


In [20]:
gc.collect()

92

In [21]:
c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=6000,max_depth=18,n_estimators=10,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=1)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

[1]	valid_0's binary_logloss: 0.686787
Training until validation scores don't improve for 1 rounds.
[2]	valid_0's binary_logloss: 0.679829
[3]	valid_0's binary_logloss: 0.672683
[4]	valid_0's binary_logloss: 0.665694
[5]	valid_0's binary_logloss: 0.66001
[6]	valid_0's binary_logloss: 0.653242
[7]	valid_0's binary_logloss: 0.646922
[8]	valid_0's binary_logloss: 0.640388
[9]	valid_0's binary_logloss: 0.634009
[10]	valid_0's binary_logloss: 0.627772
Did not meet early stopping. Best iteration is:
[10]	valid_0's binary_logloss: 0.627772


In [22]:
score=c.score(x_test,y_test)*100
print(score)

91.9585531910661


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [23]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/liviano v4.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/liviano v4.pkl']

In [33]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
42956,53813,1112445488,5qDRow,MASC,38.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.000000,0.0,35.0,2.0
32788,64997,1112458662,BmZDmVv,MASC,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.083333,2.0,3.0,24.0
10658,88310,1112467005,W9A2vVa,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,False,True,False,1.000000,3.0,17.0,11.0
78649,25975,1112335409,ZMj1lR,MASC,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,0.000000,0.0,18.0,2.0
9121,86064,1112466706,LNPZ3Bk,MASC,27.0,-,-,-,-,Graduado,...,False,False,False,True,False,True,0.002370,63.0,187.0,140.0
70849,55372,1112447652,5qp5kw,MASC,39.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.000000,1.0,22.0,19.0
43264,13473,1112039522,vVj6ADx,MASC,26.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.000000,70.0,100.0,73.0
55847,5560,1111564025,1GB1zQ,MASC,27.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,0.000000,2.0,0.0,4.0
10024,68296,1112460632,X9lzPzb,FEM,22.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,1.000000,0.0,17.0,14.0
58781,22026,1112281426,NzrzkBV,MASC,38.0,-,-,-,-,-,...,False,True,True,False,False,False,0.125000,5.0,29.0,23.0


In [34]:
x_final=prediccion[features]
y_final=c.predict(x_final)
y_final

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [35]:
prediccion['sepostulo']=y_final

In [36]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,0.0,0.0,18.0,0.0,0
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,0.0,5.0,60.0,0.0,0
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,0.0,2.0,19.0,1.0,0
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,0.0,0.0,21.0,0.0,0
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,0.0,1.0,15.0,1.0,0


In [37]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [38]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/liviano v4 bin.csv",index=False)

In [39]:
prediccion.sepostulo.mean()

0.4435